## Importing Python Packages

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter, ScalarFormatter, MaxNLocator
import numpy as np
import time
import gc
from IPython.display import HTML
import lightgbm as lgb
from sklearn.model_selection import KFold, RepeatedKFold
import itertools
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor
import xgboost
from catboost import Pool, CatBoostRegressor
import catboost
from sklearn.neural_network import MLPRegressor

# to run the notebook in the cloud
import requests
import io

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
# is the notebook running locally or on the cloud?
# this will determine where to read the data from later
local = True 

# to determine if PCA will be used on pickup and drop-off
# one-hot-encoded features
pca_on_zones = True 

In [3]:
# some configurations for plotting

print(time.strftime('%H:%M'))

gc.enable()
%config InlineBackend.figure_format = 'retina'
plt.rc('figure', dpi=300)
plt.rc('savefig', dpi=300)
pd.set_option('display.max_columns', 100)
plt.style.use('default')
fig_size = (12,6)
big_fig_size = (18,8)
fig_fc = '#ffffff'
pc = ["#4285f4", "#db4437", "#f4b400", "#0f9d58", "#ab47bc", "#00acc1", "#ff7043", 
      "#9e9d24", "#5c6bc0", "#f06292", "#00796b", "#c2185b", "#7e57c2", "#03a9f4", 
      "#8bc34a", "#fdd835", "#fb8c00", "#8d6e63", "#9e9e9e", "#607d8b"]
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
def plot_conf(ax, xlbl='', ylbl='', t=''):
    """
    This function perform operations to produce better-looking 
    visualizations
    """
    # changing the background color of the plot
    ax.set_facecolor('#ffffff')
    # modifying the ticks on plot axes
    ax.tick_params(axis='both', labelcolor='#616161', color='#ffffff')
    ax.tick_params(axis='both', which='major', labelsize=9)
    # adding a grid and specifying its color
    # ax.grid(True, color='#4d4a50')
    ax.grid(True, color='#e9e9e9')
    # making the grid appear behind the graph elements
    ax.set_axisbelow(True)
    # hiding axes
    ax.spines['bottom'].set_color('#ffffff')
    ax.spines['top'].set_color('#ffffff') 
    ax.spines['right'].set_color('#ffffff')
    ax.spines['left'].set_color('#ffffff')
    # setting the title, x label, and y label of the plot
    ax.set_title(t, fontsize=14, color='#616161', loc='left', pad=24, fontweight='bold');
    ax.set_xlabel(xlbl, labelpad=16, fontsize=11, color='#616161', fontstyle='italic');
    ax.set_ylabel(ylbl, color='#616161', labelpad=16, fontsize=11, fontstyle='italic');
     
styles = [
    dict(selector="td, th", props=[("border", "1px solid #333"), ("padding", "6px")]),
    dict(selector="td", props=[('background', '#fdf6e3')]),
    dict(selector="th.col_heading", props=[("background", "#002b36"), 
                                           ("color", "#b58900"), ("padding", "10px 16px")]),
    dict(selector="th.index_name", props=[("background", "#002b36"), 
                                          ("color", "#268bd2"), ("padding", "10px 16px")]),
    dict(selector="th.blank", props=[("background", "#002b36"), 
                                     ("color", "#268bd2"), ("padding", "10px 16px")]),
    dict(selector="th.row_heading.level0", props=[("background", "rgba(133, 153, 0, 0.1)")]),
    dict(selector="th.row_heading.level1", props=[("background", "rgba(42, 161, 152, 0.1)")]),
    dict(selector="thead tr:nth-child(2) th", props=[("border-bottom", "3px solid #333333")]),
    dict(selector="td:hover", props=[("font-weight", "bold"), 
                                     ("background", "#002b36"), ("color", "Gold")]),
]

def style_datfr(datfr):
    display(datfr.style.set_table_styles(styles).set_table_attributes(
        'style="border-collapse: collapse; border: 1px solid black"')
            .format('{:,}').set_caption(''))

if local:
    # change this to your desired location to save resulting visulaizations there
    rep_files_p = '/Users/ammar/Documents/CS-and-work/UM MDatSc/Research Project/Report-Files/'
else:
    rep_files_p = './'

01:44


## Reading the Data

In [4]:
# if running the notebook on local machine
# change the file paths to the paths on your machine

if local:
    data = pd.read_csv('./yellow_tripdata_2017-03_processed.csv', 
                       parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

    test_data = pd.read_csv('./yellow_tripdata_2018-03_processed.csv', 
                            parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

    weather_data = pd.read_csv('./NY_central_park_weather_2017-03_processed.csv', 
                               parse_dates=['Date'])

    weather_test_data = pd.read_csv('./NY_central_park_weather_2018-03_processed.csv', 
                                    parse_dates=['Date'])

    zones_lookup_table = pd.read_csv('./NYC_taxi_data/additional_files/taxi_zone_lookup.csv')

# if running the notebook on the cloud; can be used when running locally too
else:
    base_url = 'https://storage.googleapis.com/research_project_um/'
    res = requests.get('{}yellow_tripdata_2017-03_processed.csv'.format(base_url))
    res_f = io.BytesIO(res.content)
    data = pd.read_csv(res_f, parse_dates=
                       ['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

    res = requests.get('{}yellow_tripdata_2018-03_processed.csv'.format(base_url))
    res_f = io.BytesIO(res.content)
    test_data = pd.read_csv(res_f, parse_dates=
                            ['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

    res = requests.get('{}NY_central_park_weather_2017-03_processed.csv'.format(base_url))
    res_f = io.BytesIO(res.content)
    weather_data = pd.read_csv(res_f, parse_dates=['Date'])

    res = requests.get('{}NY_central_park_weather_2018-03_processed.csv'.format(base_url))
    res_f = io.BytesIO(res.content)
    weather_test_data = pd.read_csv(res_f, parse_dates=['Date'])

    res = requests.get('{}taxi_zone_lookup.csv'.format(base_url))
    res_f = io.BytesIO(res.content)
    zones_lookup_table = pd.read_csv(res_f)

### Keeping only the data of the first five days of each month

In [5]:
data = data[(data.tpep_pickup_datetime.dt.day.isin([1,4,5])) & 
            (data.tpep_pickup_datetime.dt.hour.between(13,23))].reset_index(drop=True)

test_data = test_data[(test_data.tpep_pickup_datetime.dt.day.isin([1,4,5])) & 
                      (test_data.tpep_pickup_datetime.dt.hour.between(13,23))].reset_index(drop=True)

weather_data = weather_data[(weather_data.Date.dt.day.isin([1,4,5]))].reset_index(drop=True)

weather_test_data = weather_test_data[(weather_test_data.Date.dt.day.isin([1,4,5]))].reset_index(drop=True)

In [6]:
data.shape, test_data.shape

((611114, 18), (522170, 18))

In [7]:
weather_data.shape, weather_test_data.shape

((3, 10), (3, 10))

In [8]:
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration
0,1,2017-03-01 13:00:00,2017-03-01 13:10:12,1,2.25260,1,N,236,237,2,8.5,0.0,0.5,0.00,0.00,0.3,9.30,10.200000
1,1,2017-03-01 13:00:00,2017-03-01 13:48:34,1,35.07620,2,N,132,142,2,52.0,0.0,0.5,0.00,5.54,0.3,58.34,48.566667
2,1,2017-03-01 13:00:00,2017-03-01 13:11:20,1,1.60900,1,N,234,170,1,8.5,0.0,0.5,1.85,0.00,0.3,11.15,11.333333
3,2,2017-03-01 13:00:00,2017-03-01 13:00:55,1,0.33789,1,N,263,236,2,3.0,0.0,0.5,0.00,0.00,0.3,3.80,0.916667
4,2,2017-03-01 13:00:00,2017-03-01 13:08:57,5,1.38374,1,N,43,161,1,7.0,0.0,0.5,1.56,0.00,0.3,9.36,8.950000


In [9]:
weather_data.head()

,Date,Maximum Temperature,Minimum Temperature,Average Temperature,Departure Temperature,HDD,CDD,Precipitation,New Snow,Snow Depth
0,2017-03-01,21.111111,12.222222,16.666667,-4.500000,3,0,0.3048,0.0,0.0
1,2017-03-04,-1.111111,-8.333333,-4.722222,-26.277778,41,0,0.0000,0.0,0.0
2,2017-03-05,2.777778,-10.000000,-3.611111,-25.333333,39,0,0.0000,0.0,0.0


## Feature Selection

In [10]:
data[data.RatecodeID != 99].shape[0], test_data[test_data.RatecodeID != 99].shape[0]

(611114, 522170)

In [11]:
# deleting records with missing rate-code ID (i.e. where
# rate-code ID is equal to 99)
data = data[data.RatecodeID != 99]
test_data = test_data[test_data.RatecodeID != 99]

In [12]:
data.shape[0], test_data.shape[0]

(611114, 522170)

In [13]:
(data[(data.PULocationID >= 264) | (data.DOLocationID >= 264)].shape[0], 
 test_data[(test_data.PULocationID >= 264) | (test_data.DOLocationID >= 264)].shape[0])

(12707, 9081)

In [14]:
zones_lookup_table[zones_lookup_table.LocationID >= 264]

,LocationID,Borough,Zone,service_zone
263,264,Unknown,NV,NaN
264,265,Unknown,NaN,NaN


In [15]:
# deleting records that contain location ID 264 or 265 because they
# represent erroneous locations
data = data[(~data.PULocationID.isin([264, 265])) & (~data.DOLocationID.isin([264, 265]))]
test_data = test_data[(~test_data.PULocationID.isin([264, 265])) & 
                      (~test_data.DOLocationID.isin([264, 265]))]

In [16]:
data.shape[0], test_data.shape[0]

(598407, 513089)

In [17]:
data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

### Dropping irrelevant columns

In [18]:
cols_to_drop = ['tpep_dropoff_datetime', 'trip_distance', 'RatecodeID', 
                'store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra', 
                'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 
                'total_amount']

for df in (data, test_data):
    df.drop(cols_to_drop, axis=1, inplace=True)
    
for df in (weather_data, weather_test_data):
    df.drop(['Departure Temperature', 'HDD', 'CDD'], axis=1, inplace=True)

In [19]:
print(data.shape)
data.head()

(598407, 6)


,VendorID,tpep_pickup_datetime,passenger_count,PULocationID,DOLocationID,trip_duration
0,1,2017-03-01 13:00:00,1,236,237,10.200000
1,1,2017-03-01 13:00:00,1,132,142,48.566667
2,1,2017-03-01 13:00:00,1,234,170,11.333333
3,2,2017-03-01 13:00:00,1,263,236,0.916667
4,2,2017-03-01 13:00:00,5,43,161,8.950000


In [20]:
print(weather_data.shape)
weather_data.head()

(3, 7)


,Date,Maximum Temperature,Minimum Temperature,Average Temperature,Precipitation,New Snow,Snow Depth
0,2017-03-01,21.111111,12.222222,16.666667,0.3048,0.0,0.0
1,2017-03-04,-1.111111,-8.333333,-4.722222,0.0000,0.0,0.0
2,2017-03-05,2.777778,-10.000000,-3.611111,0.0000,0.0,0.0


## Feature Engineering

### Extract features from datetime fields

In [21]:
# used to create a feature below
is_weekend_dict = {'Monday': 'No', 'Tuesday': 'No', 'Wednesday': 'No', 
                   'Thursday': 'No', 'Friday': 'No', 'Saturday': 'Yes', 
                   'Sunday': 'Yes'}

# performing the operations on training and test data
for df in [data, test_data]:
    df['pickup_day_of_month'] = df.tpep_pickup_datetime.dt.day
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_period_of_day'] = pd.cut(
        df.tpep_pickup_datetime.dt.hour, [0,2,5,8,11,14,17,20,23], include_lowest=True,
        labels=['0_2','3_5','6_8','9_11','12_14','15_17','18_20','21_23'])
    df['pickup_day_name'] = df.tpep_pickup_datetime.dt.day_name()
    df['pickup__is_weekend'] = df.pickup_day_name.map(is_weekend_dict)
    # deleting the original field 
    df.drop('tpep_pickup_datetime', axis=1, inplace=True)

In [22]:
data.head()

,VendorID,passenger_count,PULocationID,DOLocationID,trip_duration,pickup_day_of_month,pickup_hour,pickup_period_of_day,pickup_day_name,pickup__is_weekend
0,1,1,236,237,10.200000,1,13,12_14,Wednesday,No
1,1,1,132,142,48.566667,1,13,12_14,Wednesday,No
2,1,1,234,170,11.333333,1,13,12_14,Wednesday,No
3,2,1,263,236,0.916667,1,13,12_14,Wednesday,No
4,2,5,43,161,8.950000,1,13,12_14,Wednesday,No


### Temperature features

In [23]:
for (df, w_df) in [(data, weather_data), (test_data, weather_test_data)]:
    day_temp = {}
    for d, t in zip(w_df.Date.dt.day, w_df['Average Temperature']):
        day_temp[d] = t
    df['day_avg_temperature'] = df.pickup_day_of_month.map(day_temp)
    
    day_temp_range = {}
    for d, mx_t, mn_t in zip(w_df.Date.dt.day, w_df['Maximum Temperature'], 
                    w_df['Minimum Temperature']):
        day_temp_range[d] = mx_t - mn_t
    df['day_temperature_range'] = df.pickup_day_of_month.map(day_temp_range)

### Precipitation features

In [24]:
for (df, w_df) in [(data, weather_data), (test_data, weather_test_data)]:
    day_prec = {}
    for d, p in zip(w_df.Date.dt.day, w_df['Precipitation']):
        day_prec[d] = p
    df['day_precipitation'] = df.pickup_day_of_month.map(day_prec)
    
    day_new_snow = {}
    for d, ns in zip(w_df.Date.dt.day, w_df['New Snow']):
        day_new_snow[d] = ns
    df['day_new_snow'] = df.pickup_day_of_month.map(day_new_snow)
    
    day_snow_depth = {}
    for d, sd in zip(w_df.Date.dt.day, w_df['Snow Depth']):
        day_snow_depth[d] = sd
    df['day_snow_depth'] = df.pickup_day_of_month.map(day_snow_depth)

### More features

In [25]:
for df in (data, test_data):
    # is the number of passengers > 6?
    df['is_passenger_count_gt_6'] = df['passenger_count'].apply(
        lambda x: 1 if x > 6 else 0)

#### Borough and servie-zone features

In [26]:
# showing the relationship between boroughs and service zones in the 
# helper location table
style_datfr(zones_lookup_table.groupby(['Borough', 'service_zone'])
            .size().to_frame('No. of zones'))

In [27]:
locID_borough_map = {}
locID_service_zone_map = {}
for loc_id, b, sz in zip(zones_lookup_table.LocationID.tolist(), 
                         zones_lookup_table.Borough.tolist(),
                         zones_lookup_table.service_zone.tolist()):
    locID_borough_map[loc_id] = b
    locID_service_zone_map[loc_id] = sz


for df in [data, test_data]:
    df['pickup_borough'] = df['PULocationID'].map(locID_borough_map)
    df['dropoff_borough'] = df['DOLocationID'].map(locID_borough_map)
    df['pickup_service_zone'] = df['PULocationID'].map(locID_service_zone_map)
    df['dropoff_service_zone'] = df['DOLocationID'].map(locID_service_zone_map)    

### One-hot encoding and label encoding

In [28]:
data.head()

,VendorID,passenger_count,PULocationID,DOLocationID,trip_duration,pickup_day_of_month,pickup_hour,pickup_period_of_day,pickup_day_name,pickup__is_weekend,day_avg_temperature,day_temperature_range,day_precipitation,day_new_snow,day_snow_depth,is_passenger_count_gt_6,pickup_borough,dropoff_borough,pickup_service_zone,dropoff_service_zone
0,1,1,236,237,10.200000,1,13,12_14,Wednesday,No,16.666667,8.888889,0.3048,0.0,0.0,0,Manhattan,Manhattan,Yellow Zone,Yellow Zone
1,1,1,132,142,48.566667,1,13,12_14,Wednesday,No,16.666667,8.888889,0.3048,0.0,0.0,0,Queens,Manhattan,Airports,Yellow Zone
2,1,1,234,170,11.333333,1,13,12_14,Wednesday,No,16.666667,8.888889,0.3048,0.0,0.0,0,Manhattan,Manhattan,Yellow Zone,Yellow Zone
3,2,1,263,236,0.916667,1,13,12_14,Wednesday,No,16.666667,8.888889,0.3048,0.0,0.0,0,Manhattan,Manhattan,Yellow Zone,Yellow Zone
4,2,5,43,161,8.950000,1,13,12_14,Wednesday,No,16.666667,8.888889,0.3048,0.0,0.0,0,Manhattan,Manhattan,Yellow Zone,Yellow Zone


In [29]:
data.PULocationID.unique().shape[0]

223

In [30]:
# columns to be used for one-hot encoding
cols_for_onehot_enc = ['VendorID', 'PULocationID', 'DOLocationID', 'pickup_day_name',
                       'pickup_borough', 'dropoff_borough', 'pickup_service_zone', 
                       'dropoff_service_zone', 'pickup_period_of_day', 'pickup_day_of_month']

onehot_col_prefixes = {x:x for x in cols_for_onehot_enc}

# columns to be used for label encoding
cols_for_label_enc = ['pickup__is_weekend']

# columns to be used for target encoding later
cols_for_target_enc = ['PULocationID', 'DOLocationID', 'pickup_period_of_day', 'pickup_hour']

# columns for both one-hot encoding and target encoding
cols_targetend_and_onehot = [x for x in cols_for_target_enc if x in cols_for_onehot_enc]

In [31]:
# performing one-hot encoding and label encoding while keeping the
# columns that will be used for target encoding later

# one-hot encoding: training data
tmp_cols = data[cols_targetend_and_onehot]

data = pd.get_dummies(data, columns=cols_for_onehot_enc, 
                      prefix=onehot_col_prefixes, prefix_sep='__')
for c in tmp_cols:
    data[c] = tmp_cols[c].values
    
# one-hot encoding: test data
tmp_cols = test_data[cols_targetend_and_onehot]

test_data = pd.get_dummies(test_data, columns=cols_for_onehot_enc, 
                           prefix=onehot_col_prefixes, prefix_sep='__')

for c in tmp_cols:
    test_data[c] = tmp_cols[c].values
    
# label encoding
for df in (data, test_data):
    for c in cols_for_label_enc:
        df[c] = pd.factorize(df[c])[0]
        
display(data.shape, test_data.shape)

(598407, 528)

(513089, 530)

In [32]:
# align data and test_data (making sure they have the same 
# columns) after one-hot encoding
missing_cols = set(data.columns) - set(test_data.columns)
for c in missing_cols:
    test_data[c] = 0

missing_cols = set(test_data.columns) - set(data.columns)
for c in missing_cols:
    data[c] = 0
    
# to ensure the same column order also
test_data = test_data[data.columns]

In [33]:
data.shape, test_data.shape

((598407, 542), (513089, 542))

### Performing PCA on location-zones (i.e. location-IDs) features

In [34]:
if pca_on_zones:
    n_comp = 130
    locid_cols = [c for c in data.columns 
                  if (c.startswith('PULocationID__') or c.startswith('DOLocationID__'))]
    tmp_data = data[locid_cols]
    data.drop(locid_cols, axis=1, inplace=True)
    pca = PCA(random_state=7, svd_solver='randomized', n_components=n_comp).fit(tmp_data)
    tmp_cls = ['PUDOLocIdPCA_{}'.format(x) for x in range(1,n_comp+1)]
    tmp_data = pd.DataFrame(pca.transform(tmp_data), columns=tmp_cls)
    data = pd.concat([data, tmp_data], axis=1)

    locid_cols = [c for c in test_data.columns 
                  if (c.startswith('PULocationID__') or c.startswith('DOLocationID__'))]
    tmp_data = test_data[locid_cols]
    test_data.drop(locid_cols, axis=1, inplace=True)
    tmp_cls = ['PUDOLocIdPCA_{}'.format(x) for x in range(1,n_comp+1)]
    tmp_data = pd.DataFrame(pca.transform(tmp_data), columns=tmp_cls)
    test_data = pd.concat([test_data, tmp_data], axis=1)

### Adding more features after feature encoding

In [35]:
# features that indicate the origin and destination boroughs of the trip
for df in (data, test_data):
    cols_pickup_bor = [c for c in df.columns if c.startswith('pickup_borough__')]
    cols_dropoff_bor = [c for c in df.columns if c.startswith('dropoff_borough__')]
    for cp, cd in itertools.product(cols_pickup_bor, cols_dropoff_bor):
        df[cp + '_x_' + cd] = df[cp] * df[cd]

In [36]:
data.shape, test_data.shape

((598407, 217), (513089, 217))

### Removing features that have one unique value only for all trips

In [37]:
cols_with_one_val = []
for c in data.columns:
    if (data[c].unique().shape[0] == 1) and (test_data[c].unique().shape[0] == 1):
        cols_with_one_val.append(c)
data.drop(cols_with_one_val, axis=1, inplace=True)
test_data.drop(cols_with_one_val, axis=1, inplace=True)

In [38]:
data.shape, test_data.shape

((598407, 202), (513089, 202))

## Model Selection: Cross Validation

In [39]:
def CV_helper(train_index, val_index):
    global data, cols_for_target_enc
    
    # creating datasets for cross validation
    X_tr = data.iloc[train_index]
    X_val = data.iloc[val_index] 
    y_tr = data.trip_duration.iloc[train_index]
    y_val = data.trip_duration.iloc[val_index]
    
    # target encoding
    for col in cols_for_target_enc:
        means = X_tr.groupby(col).trip_duration.mean()
        X_tr[col + '_mean_encoded'] = X_tr[col].map(means)
        X_val[col + '_mean_encoded'] = X_val[col].map(means)
        
        medians = X_tr.groupby(col).trip_duration.agg(lambda ser: np.median(ser))
        X_tr[col + '_median_encoded'] = X_tr[col].map(medians)
        X_val[col + '_median_encoded'] = X_val[col].map(medians)
        
        stds = X_tr.groupby(col).trip_duration.std(ddof=0)
        X_tr[col + '_std_encoded'] = X_tr[col].map(stds)
        X_val[col + '_std_encoded'] = X_val[col].map(stds)
        
        mins = X_tr.groupby(col).trip_duration.min()
        X_tr[col + '_min_encoded'] = X_tr[col].map(mins)
        X_val[col + '_min_encoded'] = X_val[col].map(mins)
        
        maxs = X_tr.groupby(col).trip_duration.max()
        X_tr[col + '_max_encoded'] = X_tr[col].map(maxs)
        X_val[col + '_max_encoded'] = X_val[col].map(maxs)
        
        X_tr.drop(col, axis=1, inplace=True)
        X_val.drop(col, axis=1, inplace=True)    
    
    X_tr = X_tr.drop('trip_duration', axis=1)
    X_val = X_val.drop('trip_duration', axis=1)
    
    target_endoded_vars = [x for x in X_val.columns if 
                           x.endswith(('_mean_encoded', '_median_encoded', 
                                       '_std_encoded', '_min_encoded', 
                                       '_max_encoded'))]
    
    X_val[target_endoded_vars] = X_val[target_endoded_vars].fillna(-1)
    
    return (X_tr, y_tr, X_val, y_val)

In [40]:
# LightGBM model

print(time.strftime('%H:%M'))

kf = KFold(n_splits=2, shuffle=True, random_state=7) 

mae_cv = []

for train_index, val_index in kf.split(data.drop('trip_duration', axis=1)):
    
    # creating the datasets for CV with target encoding
    X_tr, y_tr, X_val, y_val = CV_helper(train_index, val_index)
    
    # creating and running the model
    train_data = lgb.Dataset(X_tr, label=y_tr)
    
    params={'learning_rate': 0.1, 'objective':'regression_l1', 
            'metric':'', 'num_leaves': 31, 'boosting': 'gbdt',
            'verbose': 1, 'random_state':311, 'max_depth': -1,
            'bagging_freq': 0, 'bagging_fraction': 1.0, 
            'feature_fraction': 1.0, 'min_data_in_leaf': 20, 
            'num_threads': 2, 'verbosity': -1, 
            'early_stopping_round': 15}
    
    num_round = 500
    
    light = lgb.train(params, train_data, num_round, valid_sets=[lgb.Dataset(X_val, label=y_val)], 
                      valid_names=['validation'], early_stopping_rounds=15, verbose_eval=False)
    
    light_pred = light.predict(X_val)
    
    print('Iteration score: ', end='')
    print(mean_absolute_error(light_pred, y_val))
    
    mae_cv.append(mean_absolute_error(light_pred, y_val))

print('Average score across iterations =', np.mean(mae_cv))

gc.collect(); print(time.strftime('%H:%M'))

01:47
Iteration score: 3.096901162030756
Iteration score: 3.1011358083065783
Average score across iterations = 3.099018485168667
01:50


In [41]:
# XGBoost model

print(time.strftime('%H:%M'))

kf = KFold(n_splits=2, shuffle=True, random_state=7) 

mae_cv = []

for train_index, val_index in kf.split(data.drop('trip_duration', axis=1)):
    
    # creating the datasets for CV with target encoding
    X_tr, y_tr, X_val, y_val = CV_helper(train_index, val_index)
    
    # creating and running the model
    dtrain = xgboost.DMatrix(X_tr, label=y_tr)
    dvalid = xgboost.DMatrix(X_val)
    
    params = {"seed": 3, "learning_rate": 0.1, "max_depth": 6, "eval_metric": "mae",
              "objective": "reg:squarederror", "min_child_weight": 1, "subsample": 1,
              "colsample_bytree": 1, "colsample_bylevel": 1, "lambda": 1, "alpha": 0,
              "nthread": 2, "verbosity": 0} 
    
    num_round = 500
    
    xgb = xgboost.train(params, dtrain, num_round, early_stopping_rounds=15,
                        evals=[(xgboost.DMatrix(X_val, label=y_val), 'val')],
                        verbose_eval=False)
    
    xgb_pred = xgb.predict(dvalid)
    
    print('Iteration score: ', end='')
    print(mean_absolute_error(xgb_pred, y_val))
    
    mae_cv.append(mean_absolute_error(xgb_pred, y_val))

print('Average score across iterations =', np.mean(mae_cv))

gc.collect()

print(time.strftime('%H:%M'))

01:50
Iteration score: 3.161393195196962
Iteration score: 3.1509659330606743
Average score across iterations = 3.156179564128818
02:19


In [42]:
# CatBoost model

print(time.strftime('%H:%M'))

kf = KFold(n_splits=2, shuffle=True, random_state=7) 

mae_cv = []

for train_index, val_index in kf.split(data.drop('trip_duration', axis=1)):
    
    # creating the datasets for CV with target encoding
    X_tr, y_tr, X_val, y_val = CV_helper(train_index, val_index)
    
    # creating and running the model
    train_pool = Pool(X_tr, label=y_tr)
    
    valid_pool = Pool(X_val)
    
    cat = CatBoostRegressor(loss_function='MAE', random_seed=7, 
                            eval_metric='MAE', max_depth=6, 
                            learning_rate=0.05, reg_lambda=3,
                            bootstrap_type='Bayesian', bagging_temperature=0.5,
                            iterations=500, od_type='Iter', od_wait=15,
                            thread_count=2, logging_level='silent')
    
    cat.fit(train_pool, eval_set=Pool(X_val, label=y_val), 
            early_stopping_rounds=15, verbose=False)
    
    cat_pred = cat.predict(valid_pool)
    
    print('Iteration score: ', end='')
    print(mean_absolute_error(cat_pred, y_val))
    
    mae_cv.append(mean_absolute_error(cat_pred, y_val))

print('Average score across iterations =', np.mean(mae_cv))

gc.collect()

print(time.strftime('%H:%M'))

02:19
Iteration score: 6.752881564778302
Iteration score: 6.748844001519481
Average score across iterations = 6.750862783148891
02:24


In [43]:
# Random-Forest model

print(time.strftime('%H:%M'))

kf = KFold(n_splits=2, shuffle=True, random_state=7) 

mae_cv = []

for train_index, val_index in kf.split(data.drop('trip_duration', axis=1)):
    
    # creating the datasets for CV with target encoding
    X_tr, y_tr, X_val, y_val = CV_helper(train_index, val_index)
    
    # creating and running the model
    rf = RandomForestRegressor(n_estimators=50, criterion="mse", max_depth=7, 
                               min_samples_split=100, min_samples_leaf=100, 
                               min_weight_fraction_leaf=0.0, max_features=1.0, 
                               max_leaf_nodes=None, min_impurity_decrease=0.0, 
                               min_impurity_split=None, bootstrap=True, oob_score=False, 
                               n_jobs=2, random_state=311, verbose=1, warm_start=False)
    
    rf.fit(X_tr, y_tr)
    
    rf_pred = rf.predict(X_val)
    
    print('Iteration score: ', end='')
    print(mean_absolute_error(rf_pred, y_val))
    
    mae_cv.append(mean_absolute_error(rf_pred, y_val))

print('Average score across iterations =', np.mean(mae_cv))

gc.collect()

print(time.strftime('%H:%M'))

02:24


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  3.3min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:  3.6min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.5s finished


Iteration score: 4.252739668357832


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  3.3min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:  3.6min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.5s finished


Iteration score: 4.27479039289201
Average score across iterations = 4.263765030624921
02:32


In [44]:
# Neural-Network model

print(time.strftime('%H:%M'))

kf = KFold(n_splits=2, shuffle=True, random_state=7) 

mae_cv = []

for train_index, val_index in kf.split(data.drop('trip_duration', axis=1)):
    
    # creating the datasets for CV with target encoding
    X_tr, y_tr, X_val, y_val = CV_helper(train_index, val_index)
    
    # creating and running the model
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_val = scaler.transform(X_val)

    nn = MLPRegressor(hidden_layer_sizes=(int(X_tr.shape[1]/2),), activation="relu",
                      solver='adam', alpha=1e-5,
                      batch_size=200, learning_rate="constant",
                      learning_rate_init=0.01,
                      power_t=0.5, max_iter=50, shuffle=False,
                      random_state=7, tol=1e-4,
                      verbose=True, warm_start=False, momentum=0.9,
                      nesterovs_momentum=True, early_stopping=False,
                      validation_fraction=0, beta_1=0.9, beta_2=0.999,
                      epsilon=1e-8, n_iter_no_change=15)

    nn.fit(X_tr, y_tr)
    
    nn_pred = nn.predict(X_val)

    print('Iteration score: ', end='')
    print(mean_absolute_error(nn_pred, y_val))
    
    mae_cv.append(mean_absolute_error(nn_pred, y_val))

print('Average score across iterations =', np.mean(mae_cv))

gc.collect()

print(time.strftime('%H:%M'))

02:32
Iteration 1, loss = 30.03207962
Iteration 2, loss = 26.67565668
Iteration 3, loss = 24.90018379
Iteration 4, loss = 24.15926252
Iteration 5, loss = 23.23774605
Iteration 6, loss = 22.22435254
Iteration 7, loss = 19.40402694
Iteration 8, loss = 16.54543473
Iteration 9, loss = 15.37557970
Iteration 10, loss = 14.96704837
Iteration 11, loss = 14.62999882
Iteration 12, loss = 14.44863191
Iteration 13, loss = 14.37030320
Iteration 14, loss = 14.13474269
Iteration 15, loss = 13.98960332
Iteration 16, loss = 13.83165225
Iteration 17, loss = 13.75067624
Iteration 18, loss = 13.50515331
Iteration 19, loss = 13.47445567
Iteration 20, loss = 13.34652783
Iteration 21, loss = 13.38503203
Iteration 22, loss = 13.29724049
Iteration 23, loss = 13.22200343
Iteration 24, loss = 13.15256371
Iteration 25, loss = 13.09165153
Iteration 26, loss = 13.10945037
Iteration 27, loss = 13.11290134
Iteration 28, loss = 13.05233293
Iteration 29, loss = 12.97473850
Iteration 30, loss = 12.97864252
Iteration 31,

In [45]:
# K-Nearest_Neighbors model

print(time.strftime('%H:%M'))

kf = KFold(n_splits=2, shuffle=True, random_state=7) 

mae_cv = []

for train_index, val_index in kf.split(data.drop('trip_duration', axis=1)):
    
    # creating the datasets for CV with target encoding
    X_tr, y_tr, X_val, y_val = CV_helper(train_index, val_index)
    
    # creating and running the model
    scaler = StandardScaler()
    scaler.fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_val = scaler.transform(X_val)

    knn = KNeighborsRegressor(n_neighbors=5, weights='uniform',
                              algorithm='brute', leaf_size=30,
                              p=2, metric='minkowski', metric_params=None, 
                              n_jobs=2)
    knn.fit(X_tr, y_tr)
    knn_pred = knn.predict(X_val)

    print('Iteration score: ', end='')
    print(mean_absolute_error(knn_pred, y_val))
    
    mae_cv.append(mean_absolute_error(knn_pred, y_val))

print('Average score across iterations =', np.mean(mae_cv))

gc.collect()

print(time.strftime('%H:%M'))

02:59
Iteration score: 3.4555905558303595
Iteration score: 3.4538838180098463
Average score across iterations = 3.454737186920103
07:18


In [46]:
# Linear-Support-Vector-Regression model

print(time.strftime('%H:%M'))

kf = KFold(n_splits=2, shuffle=True, random_state=7) 

mae_cv = []

for train_index, val_index in kf.split(data.drop('trip_duration', axis=1)):
    
    # creating the datasets for CV with target encoding
    X_tr, y_tr, X_val, y_val = CV_helper(train_index, val_index)
    
    # creating and running the model
    scaler = StandardScaler()
    scaler.fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_val = scaler.transform(X_val)

    sv = LinearSVR(epsilon=0.0, tol=1e-4, C=1.0,
                   loss='epsilon_insensitive', fit_intercept=True,
                   intercept_scaling=1., dual=True, verbose=0,
                   random_state=3, max_iter=500)
    
    sv.fit(X_tr, y_tr)
    
    sv_pred = sv.predict(X_val)

    print('Iteration score: ', end='')
    print(mean_absolute_error(sv_pred, y_val))
    
    mae_cv.append(mean_absolute_error(sv_pred, y_val))

print('Average score across iterations =', np.mean(mae_cv))

gc.collect()

print(time.strftime('%H:%M'))

07:18
Iteration score: 5.561431322146298
Iteration score: 5.5654361221413415
Average score across iterations = 5.56343372214382
07:22


In [47]:
# SGDRegressor model

print(time.strftime('%H:%M'))

kf = KFold(n_splits=2, shuffle=True, random_state=7) 

mae_cv = []

for train_index, val_index in kf.split(data.drop('trip_duration', axis=1)):
    
    # creating the datasets for CV with target encoding
    X_tr, y_tr, X_val, y_val = CV_helper(train_index, val_index)
    
    # creating and running the model
    scaler = StandardScaler()
    scaler.fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_val = scaler.transform(X_val)

    sgd = SGDRegressor(loss="huber", penalty="l1", alpha=0.001,
                       l1_ratio=0.15, fit_intercept=True, max_iter=500, tol=1e-3,
                       shuffle=True, verbose=0, epsilon=0.1,
                       random_state=2, learning_rate="invscaling", eta0=0.01,
                       power_t=0.25, early_stopping=True, validation_fraction=0.1,
                       n_iter_no_change=15, warm_start=False, average=False)
    
    sgd.fit(X_tr, y_tr)
    
    sgd_pred = sgd.predict(X_val)

    print('Iteration score: ', end='')
    print(mean_absolute_error(sgd_pred, y_val))
    
    mae_cv.append(mean_absolute_error(sgd_pred, y_val))

print('Average score across iterations =', np.mean(mae_cv))

gc.collect()

print(time.strftime('%H:%M'))

07:22
Iteration score: 5.5955174003082675
Iteration score: 5.603192942764122
Average score across iterations = 5.599355171536194
07:24


## Model Selection: Hyperparameter Optimization

In [48]:
# LightGBM model

print(time.strftime('%H:%M'))

# the possible parameter values that we will use in searching
# for the best combination of parameters
parameter_space = {
    'learning_rate': [0.1, 0.05, 0.01, 0.005],
    'objective': ['regression_l1', 'regression_l2', 'regression_l1', 'regression_l2', 'huber'],
    'num_leaves': [31, 55, 77, 100, 150],
    'max_depth': [6, 7, 8, 9, 11, -1],
    'min_data_in_leaf': [20, 50, 100, 200, 500, 1000, 2000],
    'bagging_freq': [0, 1, 2, 3],
    'bagging_fraction': [0.7, 0.8, 0.9, 1.0, 1.0],
    'feature_fraction': [0.7, 0.8, 0.9, 1.0, 1.0],
    'num_round': [300, 500, 700, 1000, 1200]
}

for i in range(20):
    
    print('### {} ###'.format(i+1).center(50))

    kf = KFold(n_splits=2, shuffle=True, random_state=7) 

    mae_cv = []
    
    params={'learning_rate': np.random.choice(parameter_space['learning_rate']), 
            'objective': np.random.choice(parameter_space['objective']), 
            'num_leaves': np.random.choice(parameter_space['num_leaves']), 
            'max_depth': np.random.choice(parameter_space['max_depth']),
            'min_data_in_leaf': np.random.choice(parameter_space['min_data_in_leaf']),
            'bagging_freq': np.random.choice(parameter_space['bagging_freq']),
            'bagging_fraction': np.random.choice(parameter_space['bagging_fraction']), 
            'feature_fraction': np.random.choice(parameter_space['feature_fraction']), 
            'verbosity': -1, 'verbose':-1, 'random_state':311, 'early_stopping_round': 15,
            'metric':'', 'num_threads': 2, 'boosting': 'gbdt'}

    num_round = np.random.choice(parameter_space['num_round'])

    for train_index, val_index in kf.split(data.drop('trip_duration', axis=1)):

        # creating the datasets for CV with target encoding
        X_tr, y_tr, X_val, y_val = CV_helper(train_index, val_index)
        
        # creating and running the model
        train_data = lgb.Dataset(X_tr, label=y_tr)

        light = lgb.train(params, train_data, num_round, valid_sets=[lgb.Dataset(X_val, label=y_val)], 
                          valid_names=['validation'], early_stopping_rounds=15, verbose_eval=False)

        light_pred = light.predict(X_val)

        print('Iteration score: ', end='')
        print(mean_absolute_error(light_pred, y_val))

        mae_cv.append(mean_absolute_error(light_pred, y_val))

    print(params)
    print('num_rounds:', num_round)
    print('Average score across iterations =', np.mean(mae_cv))
    
    gc.collect()
    
    print('='*60)
    print(time.strftime('%H:%M'))    
    print('='*60, flush=True)   
    
print(time.strftime('%H:%M'))

07:24
                    ### 1 ###                     
Iteration score: 3.054869252826567
Iteration score: 3.063179381719364
{'learning_rate': 0.05, 'objective': 'regression_l1', 'num_leaves': 100, 'max_depth': 8, 'min_data_in_leaf': 20, 'bagging_freq': 1, 'bagging_fraction': 0.8, 'feature_fraction': 0.8, 'verbosity': -1, 'verbose': -1, 'random_state': 311, 'early_stopping_round': 15, 'metric': '', 'num_threads': 2, 'boosting': 'gbdt'}
num_rounds: 500
Average score across iterations = 3.059024317272965
07:27
                    ### 2 ###                     
Iteration score: 3.0727935422558077
Iteration score: 3.0674330379840473
{'learning_rate': 0.1, 'objective': 'regression_l2', 'num_leaves': 55, 'max_depth': 9, 'min_data_in_leaf': 100, 'bagging_freq': 3, 'bagging_fraction': 1.0, 'feature_fraction': 0.7, 'verbosity': -1, 'verbose': -1, 'random_state': 311, 'early_stopping_round': 15, 'metric': '', 'num_threads': 2, 'boosting': 'gbdt'}
num_rounds: 1200
Average score across iteration

                    ### 14 ###                    
Iteration score: 3.9862611467301488
Iteration score: 3.976460112777657
{'learning_rate': 0.01, 'objective': 'regression_l2', 'num_leaves': 55, 'max_depth': 6, 'min_data_in_leaf': 2000, 'bagging_freq': 0, 'bagging_fraction': 0.9, 'feature_fraction': 1.0, 'verbosity': -1, 'verbose': -1, 'random_state': 311, 'early_stopping_round': 15, 'metric': '', 'num_threads': 2, 'boosting': 'gbdt'}
num_rounds: 300
Average score across iterations = 3.981360629753903
08:28
                    ### 15 ###                    
Iteration score: 3.706120027396807
Iteration score: 3.704136970762993
{'learning_rate': 0.05, 'objective': 'huber', 'num_leaves': 77, 'max_depth': -1, 'min_data_in_leaf': 200, 'bagging_freq': 3, 'bagging_fraction': 0.8, 'feature_fraction': 1.0, 'verbosity': -1, 'verbose': -1, 'random_state': 311, 'early_stopping_round': 15, 'metric': '', 'num_threads': 2, 'boosting': 'gbdt'}
num_rounds: 500
Average score across iterations = 3.7051284

In [49]:
# XGBoost model

print(time.strftime('%H:%M'))

# the possible parameter values that we will use in searching
# for the best combination of parameters
parameter_space = {
    'learning_rate': [0.1, 0.05, 0.01, 0.005],
    'eval_metric': ['rmse', 'mae', 'mae'],
    'max_depth': [6, 7, 8, 9, 11, 15],
    'min_child_weight': [1, 2, 3, 5, 7, 15, 50],
    'subsample': [0.7, 0.8, 0.9, 1.0, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0, 1.0],
    'colsample_bylevel': [0.7, 0.8, 0.9, 1.0, 1.0],
    'lambda': [0.5, 1, 2, 3, 5, 10, 50],
    'alpha': [0.5, 1, 2, 5, 10, 50],
    'num_round': [300, 500, 700]
}

for i in range(3):
    
    print('### {} ###'.format(i+1).center(50))

    kf = KFold(n_splits=2, shuffle=True, random_state=7) 

    mae_cv = []
    
    params = {"learning_rate": np.random.choice(parameter_space['learning_rate']), 
              "eval_metric": np.random.choice(parameter_space['eval_metric']),
              "max_depth": np.random.choice(parameter_space['max_depth']), 
              "min_child_weight": np.random.choice(parameter_space['min_child_weight']), 
              "subsample": np.random.choice(parameter_space['subsample']),
              "colsample_bytree": np.random.choice(parameter_space['colsample_bytree']), 
              "colsample_bylevel": np.random.choice(parameter_space['colsample_bylevel']), 
              "lambda": np.random.choice(parameter_space['lambda']), 
              "alpha": np.random.choice(parameter_space['alpha']),
              "objective": 'reg:squarederror', "nthread": 1, "verbosity": 0, "seed": 3}

    num_round = np.random.choice(parameter_space['num_round'])

    for train_index, val_index in kf.split(data.drop('trip_duration', axis=1)):

        # creating and running the model
        dtrain = xgboost.DMatrix(X_tr, label=y_tr)
        dvalid = xgboost.DMatrix(X_val)

        xgb = xgboost.train(params, dtrain, num_round, early_stopping_rounds=15,
                            evals=[(xgboost.DMatrix(X_val, label=y_val), 'val')],
                            verbose_eval=False)

        xgb_pred = xgb.predict(dvalid)

        print('Iteration score: ', end='')
        print(mean_absolute_error(xgb_pred, y_val))

        mae_cv.append(mean_absolute_error(xgb_pred, y_val))

    print(params)
    print('num_rounds:', num_round)
    print('Average score across iterations =', np.mean(mae_cv))
    
    gc.collect()
    
    print('='*60)
    print(time.strftime('%H:%M'))    
    print('='*60)   
    
print(time.strftime('%H:%M'))

09:02
                    ### 1 ###                     
Iteration score: 3.1047926397756838
Iteration score: 3.1047926397756838
{'learning_rate': 0.005, 'eval_metric': 'mae', 'max_depth': 11, 'min_child_weight': 3, 'subsample': 1.0, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.8, 'lambda': 5.0, 'alpha': 1.0, 'objective': 'reg:squarederror', 'nthread': 1, 'verbosity': 0, 'seed': 3}
num_rounds: 700
Average score across iterations = 3.1047926397756838
10:36
                    ### 2 ###                     
Iteration score: 3.0827269194769076
Iteration score: 3.0827269194769076
{'learning_rate': 0.1, 'eval_metric': 'mae', 'max_depth': 8, 'min_child_weight': 2, 'subsample': 0.8, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.9, 'lambda': 1.0, 'alpha': 10.0, 'objective': 'reg:squarederror', 'nthread': 1, 'verbosity': 0, 'seed': 3}
num_rounds: 500
Average score across iterations = 3.0827269194769076
11:33
                    ### 3 ###                     
Iteration score: 4.277638637492